In [1]:
from tpm2_pytss import *

In [8]:
# Open a TPM context
with ESAPI() as e:
    # Define a primary key (RSA 2024, SHA256)
    inPublic = TPM2B_PUBLIC(
        publicArea=TPMT_PUBLIC(
            type=TPM2_ALG.RSA,
            nameAlg=TPM2_ALG.SHA256,
            objectAttributes=(
                TPMA_OBJECT.SIGN_ENCRYPT
                | TPMA_OBJECT.USERWITHAUTH
                | TPMA_OBJECT.SENSITIVEDATAORIGIN
            ),
            parameters=TPMU_PUBLIC_PARMS(
                rsaDetail=TPMS_RSA_PARMS(
                    scheme=TPMT_RSA_SCHEME(scheme=TPM2_ALG.NULL),
                    keyBits=2048,
                    exponent=0,
                    symmetric=TPMT_SYM_DEF_OBJECT(algorithm=TPM2_ALG.NULL),
                )
            ),
            unique=TPMU_PUBLIC_ID(rsa=TPM2B_PUBLIC_KEY_RSA()),
        )
    )

    # Create the key in the OWNER hierarchy
    handle, outPublic, _, _, _ = e.create_primary(
        primary_handle=ESYS_TR.OWNER,
        in_sensitive=TPM2B_SENSITIVE_CREATE(),
        in_public=inPublic,
        outside_info=TPM2B_DATA(),
        creation_pcr=TPML_PCR_SELECTION(),
    )

    # Get the public key
    pub_key = outPublic.publicArea.unique.rsa.buffer
    print("Clé publique (hex):", pub_key.hex())
# end with

Clé publique (hex): b0ea35b2f1854a051620f9d8c2cf1ccd2b2e84ef5dbb4f1c66d83c308e6dae8bcb250c3120a2ac70e6b30db238b7ca6e0bbe3e3937083ad4798af89d21ca8e8b9fcb018fee4909c1a67c8a2fb86ccf7d905d69ee415272ae7512126d7312562e8be4806041dfcee9011b18988b54da7e304280c15799b770ebce90ad0ecc17475d09b5e19d3c3333ad32294de76199578724f17acaa9d36a88aa496cae372bbbde37f16cf090f35fa6ec7b5d8912059c5d3380542b56f781ea731548cfbaa08d930b9c6f3acd397bf380515325bc435d29fa69e86bf7cef5cc848ade3a1fdce3e649f370e236a75dd8726d14e0f4aaff2dfe871dd8b303f5fdd674126b080b6d


In [20]:
esys = ESAPI()

TPM2_PERSISTENT_FIRST = 0x81000000
TPM2_PERSISTENT_LAST = 0x81FFFFFF

# Lister les handles persistants
more_data, capability_data = esys.get_capability(
    capability=TPM2_CAP.HANDLES,
    prop=TPM2_PERSISTENT_FIRST,
    property_count=TPM2_PERSISTENT_LAST - TPM2_PERSISTENT_FIRST + 1,
)

print("Handles persistants dans le TPM :")
for h in capability_data.data.handles:
    print(f"- Handle: {hex(h)}")
    # Conversion TPM2_HANDLE -> ESYS_TR
    tr_handle = esys.tr_from_tpmpublic(h)
    out_public, name, qualified_name = esys.read_public(tr_handle)
    print(f"  Algorithme: {out_public.publicArea.type}")
    if out_public.publicArea.type == TPM2_ALG.RSA:
        print(f"  Taille clé RSA: {out_public.publicArea.parameters.rsaDetail.keyBits}")
    elif out_public.publicArea.type == TPM2_ALG.ECC:
        print(f"  Courbe ECC: {out_public.publicArea.parameters.eccDetail.curveID}")

Handles persistants dans le TPM :
- Handle: 0x81000001
  Algorithme: rsa
  Taille clé RSA: 2048
- Handle: 0x81000002
  Algorithme: rsa
  Taille clé RSA: 2048
- Handle: 0x81010001
  Algorithme: rsa
  Taille clé RSA: 2048


In [21]:
ek_handle = esys.tr_from_tpmpublic(0x81000002)
out_public, name, qualified_name = esys.read_public(ek_handle)
print("EK publique (hex):", out_public.publicArea.unique.rsa.buffer.hex())

EK publique (hex): c2ecec2fe9d5946276e5760fc33bfe05e6c69d07eadb1f5b4a9269c0b7cf1e9f9f6035ad5e161e79b91bad318159f7e8d82a7929919dfe097c74f4b0d1e913ca4487a83f41edae5f02331e661470ab403018221018fc84bc0a0b53eb922637df53515adb254a183bada5ea791bc83ea196bde2959bae308570c2007ca4ecadb6d3b56bd18a9f49b1f089c1e1c35a16afc705bdb5571712bdee31ef7ae7071eb3cd0f458d77b3a00f56b57fa0bc8c2eb955f0f97c36dd3108c637e9c27937aa28bc8637aa31daf1e902f2ebd1dfaf1ed2cca4e519f0e8b36deb18366b8e862940f496dd92aa9875b4336e3285a123f77c08f99314fdbf377ce61bd3bc9d051015


In [29]:
from tpm2_pytss import *
import hashlib

esys = ESAPI()

msg = b"Hello from my TPM!"
digest = hashlib.sha256(msg).digest()

# Charger l’EK (persistante à 0x81000002)
ek_handle = esys.tr_from_tpmpublic(0x81000002)

# Préparer le digest TPM2
tpm_digest = TPM2B_DIGEST(buffer=digest)

# Schéma de signature : RSASSA + SHA256
in_scheme = TPMT_SIG_SCHEME(
    scheme=TPM2_ALG.RSASSA,
    details=TPMU_SIG_SCHEME(
        rsassa=TPMS_SIG_SCHEME_RSASSA(hashAlg=TPM2_ALG.SHA256)   # 👈 ici un objet structuré
    )
)

# Tenter une signature avec l’EK
try:
    signature = esys.sign(
        ek_handle,
        digest=tpm_digest,
        inScheme=in_scheme,
        validation=None
    )
    print("Signature brute (hex):", signature.signature.rsassa.sig.hex())
except TSS2_Exception as e:
    print("⚠️ Le TPM refuse de signer avec l’EK :", e)


NameError: name 'TPMS_SIG_SCHEME_RSASSA' is not defined